In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install flask flask-ngrok torch transformers diffusers peft ngrok flask_cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import flask
from flask import Flask, request, send_file, jsonify
import torch
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
import io
from huggingface_hub import login
import subprocess
from flask_cors import CORS


app = Flask(__name__)
CORS(app)

# Style presets mapping
STYLE_PRESETS = {
    "realistic": "realistic photography, 8k, ultra detailed",
    "cartoon": "cartoon style, vibrant colors",
    "3d": "3D render, blender, octane render",
    "minimalist": "minimalist design, simple, clean lines",
    "watercolor": "watercolor painting style"
}

# Format to dimensions mapping
FORMAT_DIMENSIONS = {
    "square": (512, 512),
    "portrait": (512, 640),
    "landscape": (1024, 576),
    "story": (576, 1024)
}

def install_dependencies():
    subprocess.run(['pip', 'install', 'pyngrok', 'flask', 'torch', 'diffusers', 'accelerate', 'peft'], check=True)

install_dependencies()

from pyngrok import ngrok

# Initialize pipeline
pipe = None

def initialize_pipeline():
    global pipe
    login(token="")
    pipe = FluxPipeline.from_pretrained(
        "black-forest-labs/FLUX.1-dev",
        torch_dtype=torch.bfloat16
    )
    pipe.enable_model_cpu_offload()

    lora_path = "/content/drive/MyDrive/AI/RenamedByFiras/flux1_dev/output/my_first_flux_lora_v1"
    pipe.load_lora_weights(lora_path, weight_name="my_first_flux_lora_v1_000001750.safetensors")

initialize_pipeline()

@app.route('/generate', methods=['POST'])
def generate():
    try:
        data = request.json

        # Validate required fields
        if not data or 'prompt' not in data:
            return jsonify({'error': 'Prompt is required'}), 400

        # Get parameters with defaults
        prompt = data['prompt']
        style = data.get('style', 'realistic')
        format = data.get('format', 'square')
        creativity = float(data.get('creativity', 50))

        # Apply style preset to prompt
        styled_prompt = f"{prompt}, {STYLE_PRESETS.get(style, '')}"

        # Get dimensions from format
        width, height = FORMAT_DIMENSIONS.get(format, (512, 512))

        # Map creativity (0-100) to guidance scale (3-5)
        guidance_scale = 3.0 + (creativity / 50.0)

        # Generate image
        image = pipe(
            styled_prompt,
            height=height,
            width=width,
            guidance_scale=guidance_scale,
            num_inference_steps=50
        ).images[0]

        # Prepare image response
        img_io = io.BytesIO()
        image.save(img_io, 'PNG')
        img_io.seek(0)

        return send_file(img_io, mimetype='image/png')

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Security Note: In production, use environment variables for these secrets
    ngrok.set_auth_token("")

    # Start ngrok tunnel
    public_url = ngrok.connect(5051).public_url
    print(f" * Running on: {public_url}")

    app.run(host='0.0.0.0', port=5051)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.53G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/19.9k [00:00<?, ?B/s]

...

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


 * Running on: https://9411-34-58-165-10.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5051
 * Running on http://172.28.0.12:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/May/2025 22:50:28] "OPTIONS /generate HTTP/1.1" 200 -
Token indices sequence length is longer than the specified maximum sequence length for this model (109 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['style should be cinematic, colorful, and dramatic, capturing action and excitement., realistic photography, 8 k, ultra detailed, realistic photography, 8 k, ultra detailed']


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [12/May/2025 22:51:20] "POST /generate HTTP/1.1" 200 -
